In [9]:
!rm -rf DNABERT-2-117M
! git lfs clone https://huggingface.co/zhihan1996/DNABERT-2-117M
%cd DNABERT-2-117M
! git fetch origin +refs/pull/*:refs/remotes/origin/pr/*
%cd ..

Cloning into 'DNABERT-2-117M'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 68 (delta 19), reused 0 (delta 0), pack-reused 31 (from 1)
Unpacking objects: 100% (68/68), done.
/home/yamlibina/DNABERT-2-117M(1/1), 468 MB | 84 MB/s                           
/home/yamlibina


In [ ]:
# !wget https://huggingface.co/zhihan1996/DNABERT-2-117M/resolve/refs%2Fpr%2F32/flash_attn_triton.py -O flash_attn_triton.py

--2025-03-03 13:47:39--  https://huggingface.co/zhihan1996/DNABERT-2-117M/resolve/refs%2Fpr%2F32/flash_attn_triton.py
Resolving huggingface.co (huggingface.co)... 3.164.240.18
Connecting to huggingface.co (huggingface.co)|3.164.240.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42721 (42K) [text/plain]
Saving to: ‘flash_attn_triton.py’

100%[======================================>] 42,721      --.-K/s   in 0.02s   

2025-03-03 13:47:39 (1.63 MB/s) - ‘flash_attn_triton.py’ saved [42721/42721]



# Imports

In [1]:
import torch

In [2]:
from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding, TrainingArguments, Trainer
from transformers.models.bert.configuration_bert import BertConfig
import numpy as np
import evaluate
import importlib

from tqdm.notebook import tqdm

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
model_path_local = "DNABERT2-6"
model_path_hug = "zhihan1996/DNABERT-2-117M"

# Loading model

In [5]:
config = BertConfig.from_pretrained(
    model_path_local,
    local_files_only=True
)

model = AutoModel.from_pretrained(
    model_path_local,
    config=config,
    trust_remote_code=True,
    local_files_only=True
)

model.to(device)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
/home/yamlibina/.cache/huggingface/modules/transformers_modules/DNABERT2-6/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of the model checkpoint at DNABERT2-6 were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected i

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(4096, 768, padding_idx=0)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertUnpadAttention(
          (self): BertUnpadSelfAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (mlp): BertGatedLinearUnitMLP(
          (gated_layers): Linear(in_features=768, out_features=6144, bias=False)
          (act): GELU(approximate='none')
          

In [6]:
module_name = model.__class__.__module__
print(module_name)

transformers_modules.DNABERT2-6.bert_layers


In [7]:
cls = getattr(importlib.import_module(module_name), 'BertForSequenceClassification')
cls

transformers_modules.DNABERT2-6.bert_layers.BertForSequenceClassification

In [8]:
model = cls.from_pretrained(model_path_local, num_labels=2)
model.to(device)

Some weights of the model checkpoint at DNABERT2-6 were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DNABERT2-6 and are newly initialized: ['classifier.weight'

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4096, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertUnpadAttention(
            (self): BertUnpadSelfAttention(
              (dropout): Dropout(p=0.0, inplace=False)
              (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (mlp): BertGatedLinearUnitMLP(
            (gated_layers): Linear(in_features=768

## Create a new tokenizer

In [10]:
import json
path_to_tokenizer = 'DNABERT2-6/tokenizer.json'
path_to_new_tokenizer = 'DNABERT2/tokenizer.json'
# We take a pre-trained DNABERT-2 tokenizer and remove tokens with length > 6 to make them more interpretable.

new_vocab = {}
new_merges = []
with open(path_to_tokenizer) as token_file:
    d = json.load(token_file)
    old_vocab = d['model']['vocab']
    for key, val in old_vocab.items():
        if (len(key) <= 6):
            new_vocab[key] = val

    merges = d['model']['merges']
    for merge in merges:
        a, b = merge.split()
        if len(a) + len(b) <= 6:
            new_merges.append(merge)

d['model']['vocab'] = new_vocab
d['model']['merges'] = new_merges

with open(path_to_tokenizer, 'w') as f:
    json.dump(d, f, indent=2)


In [11]:
print(len(old_vocab))
print(len(new_vocab))

4096
1395


# Tokenizing dataset

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_path_local)

In [12]:
from datasets import load_dataset, get_dataset_config_names
config_names = get_dataset_config_names("leannmlindsey/GUE")
print(config_names)
prom_core_all = load_dataset("leannmlindsey/GUE", name="prom_core_all")
prom_core_all
prom_core_all["test"]

['emp_H3', 'emp_H3K14ac', 'emp_H3K36me3', 'emp_H3K4me1', 'emp_H3K4me2', 'emp_H3K4me3', 'emp_H3K79me3', 'emp_H3K9ac', 'emp_H4', 'emp_H4ac', 'human_tf_0', 'human_tf_1', 'human_tf_2', 'human_tf_3', 'human_tf_4', 'mouse_0', 'mouse_1', 'mouse_2', 'mouse_3', 'mouse_4', 'prom_300_all', 'prom_300_notata', 'prom_300_tata', 'prom_core_all', 'prom_core_notata', 'prom_core_tata', 'splice_reconstructed', 'virus_covid', 'virus_species_40', 'fungi_species_20', 'EPI_K562', 'EPI_HeLa-S3', 'EPI_NHEK', 'EPI_IMR90', 'EPI_HUVEC', 'EPI_GM12878']


Dataset({
    features: ['sequence', 'label'],
    num_rows: 5920
})

In [14]:
def preprocess(data):
    tokenized = tokenizer(data["sequence"], truncation=True, padding=True, max_length=128, return_tensors='pt')
    return {key: value.squeeze(0) for key, value in tokenized.items()}

tokenized_dataset = prom_core_all.map(preprocess, batched=True)
tokenized_dataset

Map:   0%|          | 0/47356 [00:00<?, ? examples/s]

Map:   0%|          | 0/5920 [00:00<?, ? examples/s]

Map:   0%|          | 0/5920 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sequence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 47356
    })
    test: Dataset({
        features: ['sequence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5920
    })
    dev: Dataset({
        features: ['sequence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5920
    })
})

In [15]:
train_set = tokenized_dataset['train']

test_set = tokenized_dataset['test']
test_set

Dataset({
    features: ['sequence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5920
})

In [16]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training

In [17]:
def compute_metrics(eval_pred):
    print(eval_pred)
    mcc = evaluate.load("matthews_correlation")
    f1 = evaluate.load("f1")
    logits, labels = eval_pred
    logits = logits[0]

    predictions = np.argmax(logits, axis=1)
    mcc_score = mcc.compute(predictions=predictions, references=labels)["matthews_correlation"]
    f1_score = f1.compute(predictions=predictions, references=labels)["f1"]

    return {"MCC": mcc_score, "F1": f1_score}

training_args = TrainingArguments(
    output_dir="test_run",
    learning_rate=5e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_ratio=0.1,
    optim='adamw_torch',
    weight_decay=0.01,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    logging_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model='F1',
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.can_return_loss = True
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Mcc,F1
1,0.452700,0.396116,0.648347,0.825716
2,0.369900,0.460050,0.646244,0.820855
3,0.299900,0.520560,0.633143,0.816181
4,0.239300,0.475729,0.630688,0.804329
5,0.190800,0.545474,0.626270,0.803527


TrainOutput(global_step=7400, training_loss=0.31241270529257287, metrics={'train_runtime': 1740.0617, 'train_samples_per_second': 136.076, 'train_steps_per_second': 4.253, 'total_flos': 3702880745346288.0, 'train_loss': 0.31241270529257287, 'epoch': 5.0})

In [19]:
model.save_pretrained("dnabert2_coreprom_tokens6_5epochs", from_pt=True)

In [18]:
trainer.state.best_model_checkpoint

'test_run/checkpoint-1480'

# Interpretation

In [13]:
# import torch
# from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding, TrainingArguments, Trainer
# from transformers.models.bert.configuration_bert import BertConfig
# import importlib

module_name = 'transformers_modules.DNABERT-2-117M.bert_layers'
cls = getattr(importlib.import_module(module_name), 'BertForSequenceClassification')
model = cls.from_pretrained('/home/yamlibina/dnabert2_coreprom_tokens5_5epochs', num_labels=2)
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4096, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertUnpadAttention(
            (self): BertUnpadSelfAttention(
              (dropout): Dropout(p=0.0, inplace=False)
              (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (mlp): BertGatedLinearUnitMLP(
            (gated_layers): Linear(in_features=768

## Integrated gradients (transformers-interpret)

In [20]:
from transformers_interpret import SequenceClassificationExplainer
cls_explainer = SequenceClassificationExplainer(
    model,
    tokenizer)
tokens_map = {}
tokens_cnt = {}
cnt_positive = 0
cnt_truepositive = 0
for seq in tqdm(test_set):
    sequence = seq['sequence']
    label = seq['label']
    if label == 1:
        cnt_positive += 1
        word_attributions = cls_explainer(sequence)
        if label == cls_explainer.predicted_class_index:
            cnt_truepositive += 1
            for res in word_attributions:
                token, score = res
                if abs(score) > 0.05:
                    if token in tokens_map:
                        tokens_map[token].append(score)
                        tokens_cnt[token] += 1
                    else:
                        tokens_map[token] = [score]
                        tokens_cnt[token] = 1

for token in tokens_map:
    tokens_map[token] = np.mean(np.array(tokens_map[token]))


  0%|          | 0/5920 [00:00<?, ?it/s]

In [17]:
print("Number of positives in test set:", cnt_positive)
print("Number of predicted true positives on test set:", cnt_truepositive)

Number of positives in test set: 2968
Number of predicted true positives on test set: 2373


In [21]:
sorted_tokens = {k: v for k, v in sorted(tokens_map.items(), key=lambda item: abs(item[1]), reverse=True)}
i = 0
for token in sorted_tokens:
    print(token, sorted_tokens[token], tokens_cnt[token])
    i+=1
    if i == 10:
        break

TCACGA 0.8889613151550293 1
CAACAA -0.8567278385162354 1
GTATTA -0.8120841979980469 1
TACTC -0.7689669728279114 1
CGTTAA -0.7372682094573975 1
AAAATA -0.7220790982246399 1
TAATCC -0.7096239924430847 1
TTAGTT -0.7071664929389954 1
CTTATT -0.7067008018493652 2
CATAAA -0.678298830986023 1


In [22]:
sorted_tokens_cnt = {k: v for k, v in sorted(tokens_cnt.items(), key=lambda item: item[1], reverse=True)}
i = 0
for token in sorted_tokens_cnt:
    print(token, tokens_map[token], sorted_tokens_cnt[token])
    i+=1
    if i == 10:
        break

GCC 0.2160055061476042 885
CGG 0.10977953421848792 675
GCTG 0.288756774238871 522
GGA 0.23816073995182607 489
CGC 0.04811763781278084 480
GCGG 0.10707759111267205 437
GC 0.09663187000370804 429
GCA 0.10275166747203776 428
CTG 0.14888163576510693 389
GAGG 0.16821223279833794 375


In [23]:
import pandas as pd
df = pd.DataFrame(sorted_tokens.items(), columns=['token', 'score'])
df_cnt = pd.DataFrame(tokens_cnt.items(), columns = ['token', 'count'])
df = df.merge(df_cnt, on='token')
df

,token,score,count
0,TCACGA,0.888961,1
1,CAACAA,-0.856728,1
2,GTATTA,-0.812084,1
3,TACTC,-0.768967,1
4,CGTTAA,-0.737268,1
...,...,...,...
1249,CGTCA,0.000626,44
1250,TACTA,-0.000467,3
1251,TGAGGA,0.000412,10
1252,GAGGA,-0.000309,80


In [24]:
df.to_csv("/home/yamlibina/tokens_ig_interpret_all_cnt_maxlen5.csv", index=False)

In [25]:
df_pos = df[df['score'] > 0]
df_pos.to_csv("token_scores_dnabert2_promoters_max_6.csv", index=False)